# Welcome!
Below, we will learn to implement and train a policy to play atari-pong, using only the pixels as input. We will use convolutional neural nets, multiprocessing, and pytorch to implement and train our policy. Let's get started!

(I strongly recommend you to try this notebook on the Udacity workspace first before running it locally on your desktop/laptop, as performance might suffer in different environments)

In [ ]:
# render ai gym environment
#!pip install gymnasium[box2d]
import gymnasium as gym

#!pip install progressbar
import progressbar as pb

import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
#%matplotlib inline

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display
else:  
    from pyvirtualdisplay import Display
    display = Display(visible=True, size=(1400, 900))
    display.start()
    
# install package for displaying animation    
from JSAnimation.IPython_display import display_animation

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ",device) 

In [ ]:
##### NEW GYM = GYMNASIUM
#!pip install gymnasium[box2d]
#import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="rgb_array",
                                 continuous= False,
                                 gravity= -9.81,
                                 enable_wind= True,
                                 wind_power= 1.11,
                                 turbulence_power= 0.11)
            #new_step_api=False,  )

state, info = env.reset(seed = 1234)
obs = env.render()
    
done = False
while not done:
    action = env.action_space.sample()  # agent policy that uses the observation and info
    state, reward, done, trun, info = env.step(action)
    #print(action, reward)
    obs = env.render()
    plt.imshow(obs)

state_shape = env.observation_space.shape
state_size = state_shape[0]
action_size = env.action_space.n
print('State shape: ', state_size)
print('Number of actions: ', action_size)
plt.imshow(obs)

### Observation Space

The observation space is an 8-dimensional vector: 
* the coordinates of the lander (x & y), 
* its linear velocities (x & y), 
* its angle (radians), 
* its angular velocity, 
* and two booleans for whether/not each leg has ground contact.

Observation Highs:
* [1.5  1.5  5.0  5.0  3.14  5.0  True  True ]

Observation Lows:
* [-1.5  -1.5  -5.0  -5.0  -3.14  -5.0  False  False ]

Wind function:

tanh(sin(2 k (t+C)) + sin(pi k (t+C))). k is set to 0.01. C is sampled randomly between -9999 and 9999


### Action Space

There are four discrete actions available:

* 0: do nothing
* 1: fire left orientation engine
* 2: fire main engine
* 3: fire right orientation engine


### Rewards

After every step a reward is granted. The total reward of an episode is the sum of the rewards for all the steps within that episode.

For each step, the reward:

* is increased/decreased the closer/further the lander is to the landing pad.
* is increased/decreased the slower/faster the lander is moving.
* is decreased the more the lander is tilted (angle not horizontal).
* is increased by 10 points for each leg that is in contact with the ground.
* is decreased by 0.03 points each frame a side engine is firing.
* is decreased by 0.3 points each frame the main engine is firing.

The episode receives an additional reward of -100 or +100 points for crashing or landing safely respectively.

An episode is considered a solution if it scores at least 200 points.

# Utils

In [ ]:
import lunar_PPO_utils as plutils

OFF=0
MAIN=2
RIGHT=3
LEFT=1
ACTIONS=[OFF,MAIN,RIGHT,LEFT]
LOWS=np.array([-1.5, -1.5, -5., -5., -3.1415927, -5., False, False])
HIGHS=np.array([1.5, 1.5, 5., 5., 3.1415927, 5., True, True])

norm = lambda x: (x - x.mean())/x.std() if x.std()!=0. else 0.
scale = lambda x: (x - x.min())/(x.max() - x.min()) if x.max()!=x.min() else x
pix_norm = lambda x: x/255.
vals2probs = nn.Softmax(dim=1)

def scale_input(state):
    scaled_state = np.zeros_like(state)
    scaled_state[:-2] = (state[:-2] - LOWS[:-2]) / (HIGHS[:-2] - LOWS[:-2])
    scaled_state[-2:] = state[-2:] 
    return scaled_state

def scale_input_batch(states):
    scaled_batch =  np.array([scale_input(state) for state in states])     
    return scaled_batch


In [ ]:
def states_to_probs(policy, states):  
    vals = policy(torch.from_numpy(np.asarray(states)).to(device))
    probs = vals2probs(vals)
    return probs.detach().cpu().numpy()
    
def states_to_vals(policy, states):
    '''
       Params
       ======
       policy (nn)
       states (list)
       Returns: probs (as array) raw output of NN, log-probs of actions
    '''
    vals = policy(torch.from_numpy(np.asarray(states)).to(device))
    return vals.detach().cpu().numpy()

In [ ]:
#Because the full states of many of the Atari games are not completely observable from the image frames, 
#Mnih et al. “stacked” the four most recent frames so that the inputs to the network had dimension
#84⇥84⇥4. This did not eliminate partial observability for all of the games, but it was
#helpful in making many of them more Markovian.
# Text pg. 438

def prep_input(state0, state1):
    '''Takes two sequential states and returns input to the network   
        Params
        states (np.array) shape (8,)
        Returns (np.array) shape (2,8) prepped for input 
    '''
    state0 = scale_input(state0)
    state1 = scale_input(state1)
    state = np.asarray([state0, state1])
    return state

## Preprocessing
To speed up training, we can simplify the input by cropping the images and use every other pixel



In [ ]:
# show what a preprocessed image looks like
state, info = env.reset()
_, _, _, _, _ = env.step(0)
# get a frame after 20 steps
for _ in range(20):
    _, _, _, _, _ = env.step(np.random.choice([0,1,2,3]))
frame = env.render()

#fig = plt.Figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(frame)
plt.title('original, shape='+str(frame.shape))

plt.subplot(1,2,2)
#pframe = prep_img(frame)
pframe = plutils.preprocess_single(frame)
plt.title('processed, shape='+str(pframe.shape))
# 80 x 80 black and white image
plt.imshow(pframe, cmap='Greys')
plt.show()



In [ ]:
try:
    from dqn_agent import Agent

    env.seed = 1234
    agent = Agent(state_size=state_space[0], action_size=action_size, seed=env.seed)
    #agent.qnetwork_local.load_state_dict(torch.load('checkpoint_2k.pth'))

    # watch a well-trained agent
    state, _ = env.reset()
    img = plt.imshow(env.render())
    for j in range(200):
        action = agent.act(state)
        state, reward, done,trun, _ = env.step(action)
        img.set_data(env.render()) 
        plt.axis('off')
        display.display(plt.gcf())
        display.clear_output(wait=True)
        if done:
            break 

    #env.close()
except:
    pass

# Policy

## Exercise 1: Implement your policy
 
Here, we define our policy. The input is the stack of two different frames (which captures the movement), and the output is a number $P_{\rm right}$, the probability of moving left. Note that $P_{\rm left}= 1-P_{\rm right}$

In [ ]:
class Policy(nn.Module):
    ''' Estimate action probabilities from states
        state_size: number of vector observations
        action_size: number of discrete actions available
    '''
    def __init__(self, state_size=8, action_size=4, seed=1234):
        super(Policy, self).__init__()
        self.state_size = state_size
        self.action_size = action_size

        # fully connected layers
        self.state_in = nn.Linear(state_size, 64)
        #self.action_in = nn.Linear(action_size, 32) 
        self.hidden = nn.Linear(64, 16)
        self.action_out = nn.Linear(16, action_size)
        # sigmoid to logprobs
        #self.sigmoid = nn.Sigmoid()
        #self.tanh = nn.Tanh()
        #self.leaky = nn.LeakyReLU()
        self.soft = nn.Softmax(dim=1)
    
    def forward(self, state):
        ##### expecting pre-scaled values to 0,1    #around mean +/- std
        x = F.relu(self.state_in(state))
        x = F.relu(self.hidden(x))
        x = self.action_out(x)
        #return self.soft(x)
        ######## returning raw values!
        return x

In [ ]:
#seed=12345
#import lunar_PPO_utils as plutils

# use your own policy!
#policy=Policy().to(device)
policy=Policy(state_size=state_size, action_size=action_size).to(device)

# we use the adam optimizer with learning rate 2e-4
# optim.SGD is also possible
import torch.optim as optim
optimizer = optim.Adam(policy.parameters(), lr=2e-4)

In [ ]:
if False:
    x, w = env.reset()
    x1, y1, z1, u1, w1 = env.step(1)
    print("env.step(1)", scale_input(x1), y1, z1, u1, w1)
    x2, y2, z2, u2, w2 = env.step(2)
    print("env.step(2)", scale_input(x2), y2, z2, u2, w2)
    x = np.asarray([scale_input(x1), scale_input(x2)])
    p = policy(torch.from_numpy(x).to(device))
    print("policy(x)", p)
    sm = nn.Softmax(dim=1)
    sp = sm(p.detach().to('cpu'))
    print("softmax(p),", sp)

# Game visualization
pong_utils contain a play function given the environment and a policy. An optional preprocess function can be supplied. Here we define a function that plays a game and shows learning progress

In [ ]:
#plutils.play(env, policy, time=100) 
# try to add the option "preprocess=pong_utils.preprocess_single"
# to see what the agent sees

In [ ]:
### Observe untrained agent

seed=1234
steps = 100
smax = nn.Softmax(dim=1)

state, _ = env.reset(seed=seed)
frame = env.render()
frame = pix_norm(frame)
img = plt.imshow(frame)
action =  np.random.choice(ACTIONS)   
for i in range(steps):
    state1, reward1, is_done, is_trunc, info = env.step(action)
    frame1 =  env.render()
    state2, reward2, is_done, is_trunc, info = env.step(0)
    frame2 =  env.render()
    states = np.asarray([scale_input(state1),scale_input(state2)])
    vals = policy(torch.from_numpy(states).to(device))
    probs = smax(vals).detach().cpu().numpy()
    action = np.argmax(probs[-1])
    #print(action)
    reward = round((2*reward1+reward2)/3, 2)
    
    #frame = pix_norm(frame2)#      (frame1 + frame2)/2)
    frame = scale(frame2-frame1)
    img.set_data(frame)
    plt.title(str(i)+"   "+str(action)+"   "+str(reward))
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)

    if is_done or is_trunc:
        break 

# Rollout
Before we start the training, we need to collect samples. To make things efficient we use parallelized environments to collect multiple examples at once

In [ ]:
#import lunar_PPO_utils as plutils
envs = plutils.parallelEnv('LunarLander-v2', n=4, seed=1234)
vals, probs, states, actions, rewards =  plutils.collect_trajectories(envs, policy, tmax=100, skip=4)

# Function Definitions
Here you will define key functions for training. 

## Exercise 2: write your own function for training
(this is the same as policy_loss except the negative sign)

### REINFORCE
you have two choices (usually it's useful to divide by the time since we've normalized our rewards and the time of each trajectory is fixed)

1. $\frac{1}{T}\sum^T_t R_{t}^{\rm future}\log(\pi_{\theta'}(a_t|s_t))$
2. $\frac{1}{T}\sum^T_t R_{t}^{\rm future}\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)}$ where $\theta'=\theta$ and make sure that the no_grad is enabled when performing the division

### PPO
Later on, you'll implement the PPO algorithm as well, and the scalar function is given by
$\frac{1}{T}\sum^T_t \min\left\{R_{t}^{\rm future}\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)},R_{t}^{\rm future}{\rm clip}_{\epsilon}\!\left(\frac{\pi_{\theta'}(a_t|s_t)}{\pi_{\theta}(a_t|s_t)}\right)\right\}$

the ${\rm clip}_\epsilon$ function is implemented in pytorch as ```torch.clamp(ratio, 1-epsilon, 1+epsilon)```

In [ ]:
#### If policy returns raw values:

vals2probs = nn.Softmax(dim=1)

n=len(envs.ps)
envs.reset()

for _ in range(10):
    _, _, _, _ = envs.step(np.random.choice(ACTIONS,n))
    states, _, _, _ = envs.step([0]*n) #fire main engine once each

batch_input = scale_input_batch(states.copy())
vals = policy(torch.from_numpy(batch_input).to(device))
probs = vals2probs(vals).squeeze().cpu().detach().numpy()
actions = np.argmax(probs, axis=1)

states, scale_input_batch(states), vals, probs, actions
#sts = torch.stack(torch.tensor(states, dtype=torch.float32))
#policy_input = sts.view(-1,*sts.shape[-2:])
#policy(policy_input).view(states.shape[:-2])

In [ ]:
vals, probs, states, actions, rewards =  plutils.collect_trajectories(envs, policy, tmax=100, skip=4)
sts = tuple(torch.from_numpy(s) for s in states)
print(len(sts), sts[0].shape)
sts = torch.stack(sts)
print(len(states), states[0].shape, sts.shape)
policy_input = sts.view(-1, sts.shape[-1])
print(sts.shape, policy_input.shape)

In [ ]:
# clipped surrogate function
# return clipped sum of log-prob divided by T
# same thing as -policy_loss but clipped
def clipped_surrogate(policy, old_vals, 
                      states, actions, rewards,
                      discount=0.995, epsilon=0.1, beta=0.01):
    '''Clipped Surrogate Function
       
       Args:
       policy (torch.nn):
       old_vals (list of np.arrays):
       states: list of np.arrays
       actions: list of np.arrays
       rewards: list of np.arrays
       discount (np.float32):
       epsilon (np.float32):
       beta (np.float32):
       
       Returns (tensor): clipped sum of log-probs divided by T
    '''
    discount = discount**np.arange(len(rewards))
    rewards = np.asarray(rewards)*discount[:,np.newaxis]
    
    # convert rewards to future rewards
    rewards_future = rewards[::-1].cumsum(axis=0)[::-1]
    
    mean = np.mean(rewards_future, axis=1)
    std = np.std(rewards_future, axis=1) + 1.0e-10

    rewards_normalized = (rewards_future - mean[:,np.newaxis])/std[:,np.newaxis]
    
    # convert states to raw policy output
    # as log-probabilities of actions
    new_vals = states_to_vals(policy, states)
    # ratio for clipping
    ratio = new_vals/old_vals
        
    # convert everything into pytorch tensors and move to gpu if available
    #states = torch.tensor(states, dtype=torch.float, device=device)
    old_vals = torch.tensor(old_vals, dtype=torch.float, device=device)
    new_vals = torch.tensor(new_vals, dtype=torch.float, device=device)
    rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)


    # clip before applying ratio to rewards
    clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)
    clipped_surrogate = torch.min(ratio*rewards, clip*rewards)

    # include a regularization term to steer new_policy towards 0.5
    # add in 1.e-10 to avoid log(0) which gives nan
    entropy = -1*(new_vals*torch.log(old_vals+1.e-10)+(1.0-new_vals)*torch.log(1.0-old_vals+1.e-10))

    
    # returns an average of all the entries of the tensor
    # effective computing L_sur^clip / T averaged over time-step and number of trajectories
    # after all rewards have been normalized
    # This is the value to maximize.
    return torch.mean(clipped_surrogate + beta*entropy)

In [ ]:
old_vals, probs, states, actions, rewards =  plutils.collect_trajectories(envs, policy, tmax=100, skip=10)
discount=0.995
epsilon=0.1
beta=0.01
print("1:",[np.asarray(x).shape for x in [old_vals, states, actions, rewards]])


discount = discount**np.arange(len(rewards))
rewards = np.asarray(rewards)*discount[:,np.newaxis]
print("2:",discount.shape, discount[:,np.newaxis].shape, rewards.shape)

rewards_future = rewards[::-1].cumsum(axis=0)[::-1]
mean = np.mean(rewards_future, axis=1)
std = np.std(rewards_future, axis=1) + 1.0e-10
rewards_normalized = (rewards_future - mean[:,np.newaxis])/std[:,np.newaxis]
print("3:",rewards_future.shape, rewards_normalized.shape, mean.shape)

new_vals = states_to_vals(policy, states)
old_vals = torch.tensor(old_vals, dtype=torch.float, device=device)
states = torch.tensor(states, dtype=torch.float, device=device)
rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)
print("4:",[x.shape for x in [states, old_vals, new_vals]])

ratio = new_vals/old_vals
clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)
print("5:",[x.shape for x in [rewards, torch.view(rewards), ratio, clip]])

clipped_surrogate = torch.min(ratio*rewards, clip*rewards)
entropy = -1*(new_vals*torch.log(old_vals+1.e-10)+(1.0-new_vals)*torch.log(1.0-old_vals+1.e-10))
result=torch.mean(clipped_surrogate + beta*entropy)
print("6:",[x.shape for x in [clipped_surrogate, entropy, result]])


In [ ]:
old_vals, probs, states, actions, rewards =  plutils.collect_trajectories(envs, policy, tmax=100, skip=10)
discount=0.995
epsilon=0.1
beta=0.01
print("1:",[np.asarray(x).shape for x in [states, rewards]])
print([r for r in rewards])


discount = discount**np.arange(len(rewards))
rewards = np.asarray(rewards)*discount[:,np.newaxis]
print("2:",discount.shape, discount[:,np.newaxis].shape, rewards.shape)

print(discount)
print(discount[:,np.newaxis])
print(rewards)

#

In [ ]:
old_vals, probs, states, actions, rewards =  plutils.collect_trajectories(envs, policy, tmax=100, skip=10)
new_vals = states_to_vals(policy, states)
ratio = new_vals/np.asarray(old_vals)

print("1:",[x.shape for x in [ratio, np.asarray(rewards)]])
print(ratio)
print(rewards)

In [ ]:
print(rewards[0].shape, rewards[0])
print(ratio[0].shape, ratio[0])

In [ ]:
print( rewards[:,None,:] == rewards[:,None,:]*ratio)
[x.shape for x in [rewards, rewards[:,None,:], ratio, rewards[:,None,:]*ratio]]

In [ ]:
old_vals, probs, states, actions, rewards =  plutils.collect_trajectories(envs, policy, tmax=100, skip=4)

clipped_surrogate(policy, old_vals, states, actions, rewards, epsilon=0.1, beta=0.01)

# Training
We are now ready to train our policy!
WARNING: make sure to turn on GPU, which also enables multicore processing. It may take up to 45 minutes even with GPU enabled, otherwise it will take much longer!

In [ ]:
##################### PPO ###############################
#### WARNING: running through all 800 episodes will take 30-45 minutes

# training loop max iterations
episodes = 100
# episode = 800

# widget bar to display progress
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episodes).start()

envs = parallelEnv('LunarLander-v2', n=8, seed=1234)

discount_rate = .99
epsilon = 0.1
beta = .01
tmax = 320
SGD_epoch = 4

# keep track of progress
mean_rewards = []

for e in range(episodes):

    # collect trajectories
    old_vals, old_probs, states, actions, rewards = plutils.collect_trajectories(envs, policy, tmax=tmax, skip=6)
        
    total_rewards = np.sum(rewards, axis=0)

    # gradient ascent step
    for _ in range(SGD_epoch):
        
        # utilize your own clipped function!
        L = -clipped_surrogate(policy, old_vals, states, actions, rewards, epsilon=epsilon, beta=beta)

        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=.999
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=.995
    
    # get the average reward of the parallel environments
    mean_rewards.append(np.mean(total_rewards))
    
    # display some progress every 20 iterations
    if (e+1)%20 ==0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(total_rewards)))
        print(total_rewards)
        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()

In [ ]:
# save your policy!
torch.save(policy, 'PPO.policy')

# load your policy if needed
# policy = torch.load('REINFORCE_OG.policy')

# try and test out the solution!
# policy = torch.load('PPO_OG.policy')

# Results

In [ ]:
plt.plot(mean_rewards)

# Observe Gameplay

In [ ]:
# play game after training!
play(env, policy, time=100) 

In [ ]:
### Observe agent gameplay

t = 300
smax = nn.Softmax(dim=1)

state, _ = env.reset(seed=SEED)
frame = env.render()
img = plt.imshow(pix_norm(frame))
action =  np.random.choice(ACTIONS)   
for _ in range(t):
    state1, reward1, is_done, is_trunc, info = env.step(action)
    frame1 =  pix_norm(env.render())
    state2, reward2, is_done, is_trunc, info = env.step(OFF)
    frame2 =  pix_norm(env.render())
    states = np.asarray([scale_input(state1),
                         scale_input(state2)])
    probs = np.asarray([agent.act(s) for s in states])
    #vals = smax(sigs).detach().cpu().numpy()
    #probs = probs.detach().cpu().numpy()
    action = np.argmax(probs[-1])
    
    reward = reward2 + reward1/2
    
    frame = scale(frame2-frame1/2)
    img.set_data(frame)
    plt.title(str(action)+"    "+str(np.round(reward, 3)))
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)

    if is_done or is_trunc:
        break 